In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

import os


HOME_DIR = '/opt/workspace/DE-project-a/'
FIXTURE_DIR = HOME_DIR + 'fixtures/'


spark = SparkSession.\
        builder.\
        appName("pyspark-nb-1").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "file:///opt/workspace/events").\
        getOrCreate()
    
sc = spark.sparkContext
spark

In [3]:

def normalize_path(path):
    path = os.path.join(path)
    return os.path.normpath(path)


def from_xlsx(session, path, **kwargs):
    path = normalize_path(path)
    df_pandas = pd.read_excel(path, **kwargs)
    return session.createDataFrame(df_pandas)




In [ ]:
path = normalize_path(f'{FIXTURE_DIR}clients.csv')
clients = spark.read.csv(path, header=True, inferSchema=True)
clients.persist()
clients.show(20, False)
clients.printSchema()

+---------+-------------------+----------------------------+----------------------+--------------------------------------------------------+
|client_id|client_name        |client_email                |client_phone          |client_address                                          |
+---------+-------------------+----------------------------+----------------------+--------------------------------------------------------+
|1        |Douglas Mann       |malexander@example.net      |7887828319            |447 Sanders Forge Apt. 868 Port Lisafort, MT 13535      |
|2        |Linda Willis       |sherryriddle@example.org    |648.448.5400x89273    |4186 Richard Turnpike Cynthiaton, GA 06049              |
|3        |Mr. Samuel Gonzalez|adam37@example.com          |+1-727-264-1902x10293 |5333 Bradley Corner Apt. 956 Port Michael, UT 04993     |
|4        |Elijah Duke        |alexis42@example.com        |+1-913-911-7052x28833 |6919 Strong Square South Catherine, OR 42933            |
|5        |An

**Cleaning clients name from degrees and statuses**


In [ ]:
from DE-project-a/e-consultant/patterns/names import TITLES_REGEX, DEGREES_REGEX


filtered_by_name = clients \
    .withColumn('names_len', size(split(col('client_name'), r"\s+"))) \
    .filter(col('names_len') >= 3) \
    .withColumn(
        'client_name',
        regexp_replace(
            col('client_name'),
            rf"(^({TITLES_REGEX})\s*)|(\s+({DEGREES_REGEX})$)",
            "")
    )

# clients.filter(size(split(col('client_name'), r"\s+")) > 2).show(100, 100)




+---------+--------------------------+------------------------------+----------------------+-----------------------------------------------------------+
|client_id|               client_name|                  client_email|          client_phone|                                             client_address|
+---------+--------------------------+------------------------------+----------------------+-----------------------------------------------------------+
|        3|       Mr. Samuel Gonzalez|            adam37@example.com| +1-727-264-1902x10293|        5333 Bradley Corner Apt. 956 Port Michael, UT 04993|
|       14|        Patricia Mason PhD|  elizabethnichols@example.com|001-647-922-4944x04173|    8921 Gonzalez Squares Suite 970 Robertborough, CT 61170|
|       36|          Dr. Dennis Davis|         mosleydon@example.net|      226-695-0178x280|  4305 Kenneth Shores Suite 346 New Danielleshire, FL 68914|
|       39|   Dr. Gabriel Watkins DDS|           keith28@example.com|            4

In [ ]:
clients.unpersist()
spark.stop()